In [ ]:
from nnunetv2.imageio.nibabel_reader_writer import NibabelIOWithReorient
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import torch


path_to_nifti = "/media/eolika/BlueMindAI/CT/dataset/aorta2/botkin_0155/ct.nii.gz"
model_path = "/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset310_aorta_detailed/nnUNetTrainer_250epochs_NoMirroring__nnUNetResEncUNetLPlans__3d_fullres"
output_path = "/media/eolika/BlueMindAI/CT/dataset/aorta2/botkin_0155/aorta.nii.gz"

img, props = NibabelIOWithReorient().read_images([path_to_nifti])

predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=False,
    perform_everything_on_device=True,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    verbose=True,
    verbose_preprocessing=True,
    allow_tqdm=True
)

predictor.initialize_from_trained_model_folder(
    model_path,
    use_folds='all',
    checkpoint_name='checkpoint_best.pth',
)

pred, probs = predictor.predict_single_npy_array(
    input_image=img, 
    image_properties=props, 
    segmentation_previous_stage=None, 
    output_file_truncated=None, 
    save_or_return_probabilities=True
)

NibabelIOWithReorient().write_seg(
    seg=pred,
    output_fname=output_path,
    properties=props
)